# Test models

This notebook displays $S(q, \omega)$ in linear and log scale for a selected model from the QENS library. 

This model can be convoluted with a Gaussian instrument profile. 

<a id='Table of Contents'></a><h3>Table of Contents</h3>
- <a href='#imports'>Load packages</a>  
- <a href='#general_settings'>Common settings</a>  
- <a href='#choice_model'>Choice of model</a>  
- <a href='#display_model'>Plot</a>


<div class="alert alert-block alert-danger">
Only Python 3 is supported.
</div>

(<a href='#Table of Contents'>Top</a>)<a id='imports'></a><h2>Load packages</h2>

In [ ]:
from __future__ import print_function

import sys
import numpy as np
import matplotlib.pyplot as plt

import QENSmodels

# for interactive plot

from IPython.display import Javascript, display

import panel
panel.extension()
import panel.widgets as pnw

(<a href='#Table of Contents'>Top</a>)<a id='general_settings'></a><h2>Common settings</h2>

In [ ]:
# Vector of Q and omega values
q = np.arange(0.25, 2.1, 0.25)
omega = np.arange(-2, 2.01, 0.01)

# Parameters
scale = 1.0
center = 0.0
background = 0.0

(<a href='#Table of Contents'>Top</a>)<a id='choice_model'></a><h2>Choice of model</h2>

In [ ]:
options_list=['BrownianTranslationalDiffusion', 
             'JumpTranslationalDiffusion', 
             'IsotropicRotationalDiffusion',
             'WaterTeixeira']

model = pnw.ToggleGroup(options=options_list, widget_type='box',behavior='radio')

widgets_models = panel.Row("#### Models:", model)

checkbox_res = pnw.Checkbox(name='Use Gaussian resolution', height=15, value=False)

checkbox_backgd = pnw.Checkbox(name='Use flat background', height=15, value=False)

options_model = pnw.Column(widgets_models, checkbox_res, checkbox_backgd)

options_model

In [ ]:
if checkbox_res.value is True:
    sig = pnw.TextInput(
        placeholder='Sigma of Gaussian resolution',
        width=250,
        name='Sigma:')
    display(sig)
else:
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

In [ ]:
if checkbox_backgd.value is True:
    backgd = pnw.TextInput(
        placeholder='Value of flat background',
        width=250,
        name='Flat background:')
    display(backgd)
else:
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

(<a href='#Table of Contents'>Top</a>)<a id='display_model'></a><h2>Plot</h2>

In [ ]:
# Resolution    
if checkbox_res.value is True:
    sigma = float(sig.value)
    resol = np.exp(-omega**2/(2*sigma**2)) / (sigma*np.sqrt(2*np.pi))
else:
    resol = None

if model.value == 'BrownianTranslationalDiffusion':
    D = float(input("Value of self-diffusion coefficient (in A^2*meV)? "))
    model_sqw = QENSmodels.sqwBrownianTranslationalDiffusion(omega, q, scale, center, D)
    
elif model.value == 'JumpTranslationalDiffusion':
    D = float(input("Value of self-diffusion coefficient (in A^2*meV)? "))
    resTime = float(input("Residence time between jumps (in meV^-1)? "))
    model_sqw = QENSmodels.sqwJumpTranslationalDiffusion(omega, q, scale, center,
           D, resTime)
           
elif model.value == 'IsotropicRotationalDiffusion':
    R = float(input("Radius of sphere (in A)? "))
    DR = float(input("Value of rotational diffusion coefficient (in meV)? "))
    model_sqw = QENSmodels.sqwIsotropicRotationalDiffusion(omega, q, scale, center,
           R, DR) 

elif model.value == 'WaterTeixeira':
    D = float(input("Value of self-diffusion coefficient (in A^2*meV)? "))
    resTime = float(input("Residence time between jumps (in meV^-1)? "))
    R = float(input("Radius of sphere (in A)? "))
    DR = float(input("Value of rotational diffusion coefficient (in meV)? "))
    model_sqw = QENSmodels.sqwWaterTeixeira(omega, q, scale, center,
           D, resTime, R, DR)
    
if resol is not None:
    sqw = np.asarray([np.convolve(model_sqw[i,:], resol/resol.sum(), mode='same') for i in range(len(q))])
else:
    sqw = model_sqw
    
# Background
if checkbox_backgd.value is True:
    sqw += float(backgd.value)

    
# Plot
fig = plt.figure(1)


ax1 = fig.add_subplot(121)
for i in range(q.size):
    integral = np.trapz(sqw[i,:], omega)
    print ("Integral S(Q=",q[i],",w) = ", integral)
    ax1.plot(omega, sqw[i,:], label=str(q[i]))
ax1.grid(True)
ax1.set_xlabel('Omega')
ax1.set_ylabel('S(Q,w)')
ax1.set_yscale('log')

ax2 = fig.add_subplot(122)
for i in range(q.size):
    ax2.plot(omega, sqw[i,:], label=str(q[i]))
ax2.grid(True)
ax2.set_xlabel('Omega')
ax2.set_ylabel('S(Q,w)')


plt.tight_layout()
plt.suptitle('S(Q, w) log and linear scales for {model}'.format(model=model.value),x=0.55,y=1.025)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()   